In [2]:
import pandas as pd
import json
import ast
import pyarrow

In [3]:
with open('output_steam_games.json', 'r') as file:
    lines = file.readlines()

steamGamesData = [json.loads(line) for line in lines]

steamGames = pd.json_normalize(steamGamesData)

steamGamesDf = pd.DataFrame(steamGames)

In [4]:
steamGamesDf = steamGamesDf.dropna(how='all')
steamGamesDf['release_date'] = pd.to_datetime(steamGamesDf['release_date'],format = 'mixed',errors = 'coerce')
steamGamesDf['release_year'] = steamGamesDf['release_date'].dt.year.fillna(0).astype(int)
steamGamesDf.rename(columns = {'id': 'item_id'}, inplace = True)


In [5]:
steamGamesDf.loc[steamGamesDf['price'].str.contains('Free', case = False) & steamGamesDf['price'].notna(), 'price'] = 0
steamGamesDf.loc[steamGamesDf['price'].str.contains('install', case = False) & steamGamesDf['price'].notna(), 'price'] = 0
steamGamesDf.loc[steamGamesDf['price'].str.contains('play', case = False) & steamGamesDf['price'].notna(), 'price'] = 0
steamGamesDf.loc[steamGamesDf['price'].str.contains('third', case = False) & steamGamesDf['price'].notna(), 'price'] = 0
steamGamesDf.loc[steamGamesDf['price'].str.contains('449', case = False) & steamGamesDf['price'].notna(), 'price'] = 449
steamGamesDf.loc[steamGamesDf['price'].str.contains('499', case = False) & steamGamesDf['price'].notna(), 'price'] = 499

In [6]:
steamGamesGenresDf = steamGamesDf[['item_id','genres','release_year']]
steamGamesGenresDf = steamGamesGenresDf.dropna(subset = 'genres')
steamGamesGenresDf['genres'] = steamGamesGenresDf['genres'].apply(lambda x: [element.replace('&amp;', 'and') for element in x]) 

In [7]:
steamGamesGenresExplodedDf = steamGamesGenresDf.explode('genres').reset_index(drop=True)

In [9]:
steamGamesGenresDf

,item_id,genres,release_year
88310,761140,"[Action, Casual, Indie, Simulation, Strategy]",2018
88311,643980,"[Free to Play, Indie, RPG, Strategy]",2018
88312,670290,"[Casual, Free to Play, Indie, Simulation, Sports]",2017
88313,767400,"[Action, Adventure, Casual]",2017
88315,772540,"[Action, Adventure, Simulation]",2018
...,...,...,...
120439,745400,"[Action, Adventure, Casual, Indie]",2018
120440,773640,"[Casual, Indie, Simulation, Strategy]",2018
120441,733530,"[Casual, Indie, Strategy]",2018
120442,610660,"[Indie, Racing, Simulation]",2018


In [8]:
steamGamesGenresExplodedDf

,item_id,genres,release_year
0,761140,Action,2018
1,761140,Casual,2018
2,761140,Indie,2018
3,761140,Simulation,2018
4,761140,Strategy,2018
...,...,...,...
71549,610660,Indie,2018
71550,610660,Racing,2018
71551,610660,Simulation,2018
71552,658870,Casual,2017
